In [7]:
from pprint import PrettyPrinter

import pandas as pd
from IPython.display import VimeoVideo
from pymongo import MongoClient

In [8]:
# Instantiate a PrettyPrinter, and assign it to the variable pp.
pp = PrettyPrinter(indent=2)

#Create a client that connects to the database running at localhost on port 27017.
client = MongoClient(host="localhost",port=27017)

In [9]:
#Print a list of the databases available on client.
pp.pprint(list(client.list_databases()))

[ {'empty': False, 'name': 'admin', 'sizeOnDisk': 49152},
  {'empty': False, 'name': 'air-quality', 'sizeOnDisk': 9515008},
  {'empty': False, 'name': 'config', 'sizeOnDisk': 73728},
  {'empty': False, 'name': 'local', 'sizeOnDisk': 73728}]


In [10]:
db = client['air-quality'] # Assign the "air-quality" database to the variable db.
db.list_collection_names() #Use the list_collections method to print a list of the collections available in db.

['nairobi']

In [11]:
nairobi = db['nairobi'] # Assign the "nairobi" collection in db to the variable name nairobi.
nairobi.count_documents({}) # Use the count_documents method to see how many documents are in the nairobi collection.

202212

In [12]:
# Use the find_one method to retrieve one document from the nairobi collection, and assign it to the variable name result.
result = nairobi.find_one({})
pp.pprint(result)

{ 'P1': 57.8,
  '_id': ObjectId('67920c60f7e54421dbb20756'),
  'metadata': { 'lat': -1.3,
                'lon': 36.785,
                'measurement': 'P1',
                'sensor_id': 16,
                'sensor_type': 'Unknown',
                'site': 6},
  'timestamp': datetime.datetime(2018, 9, 1, 0, 0, 1)}


In [13]:
# Use the distinct method to determine how many sensor sites are included in the nairobi collection.
nairobi.distinct('metadata.site')

[6, 29]

In [14]:
# Use the count_documents method to determine how many readings there are for each site in the nairobi collection.
print("Documents from site 6:", nairobi.count_documents({'metadata.site':6}))
print("Documents from site 29:", nairobi.count_documents({'metadata.site':29}))


Documents from site 6: 70360
Documents from site 29: 131852


In [15]:
# Use the aggregate method to determine how many readings there are for each site in the nairobi collection.
result = nairobi.aggregate([{"$group":{"_id":"$metadata.site","count":{"$count":{}}}}])
pp.pprint(list(result))

[{'_id': 6, 'count': 70360}, {'_id': 29, 'count': 131852}]


In [16]:
# Use the distinct method to determine how many types of measurements have been taken in the nairobi collection.
nairobi.distinct("metadata.measurement")

['P1', 'P2', 'humidity', 'temperature']

In [17]:
# Use the find method to retrieve the PM 2.5 readings from all sites. Be sure to limit your results to 3 records only.
result = nairobi.find({"metadata.measurement":"P2"}).limit(5)
pp.pprint(list(result))

[ { 'P2': 27.72,
    '_id': ObjectId('67920c60f7e54421dbb2075b'),
    'metadata': { 'lat': -1.3,
                  'lon': 36.785,
                  'measurement': 'P2',
                  'sensor_id': 20,
                  'sensor_type': 'Unknown',
                  'site': 6},
    'timestamp': datetime.datetime(2018, 9, 1, 0, 5, 5)},
  { 'P2': 34.96,
    '_id': ObjectId('67920c60f7e54421dbb2075f'),
    'metadata': { 'lat': -1.3,
                  'lon': 36.785,
                  'measurement': 'P2',
                  'sensor_id': 35,
                  'sensor_type': 'Unknown',
                  'site': 6},
    'timestamp': datetime.datetime(2018, 9, 1, 0, 10, 8)},
  { 'P2': 35.76,
    '_id': ObjectId('67920c60f7e54421dbb20764'),
    'metadata': { 'lat': -1.3,
                  'lon': 36.785,
                  'measurement': 'P2',
                  'sensor_id': 20,
                  'sensor_type': 'Unknown',
                  'site': 6},
    'timestamp': datetime.datetime(2018, 9, 1, 0,

In [18]:
# Use the aggregate method to calculate how many readings there are for each type 
# ("humidity", "temperature", "P2", and "P1") in site 6
result = nairobi.aggregate(
[
    {"$match":{"metadata.site":6}},
    {"$group":{"_id":"$metadata.measurement","count":{"$count":{}}}}
]
)
pp.pprint(list(result))

[ {'_id': 'P2', 'count': 17551},
  {'_id': 'temperature', 'count': 17574},
  {'_id': 'humidity', 'count': 17626},
  {'_id': 'P1', 'count': 17609}]


In [19]:
# Use the aggregate method to calculate how many readings there are for each type ("humidity", "temperature", "P2", and "P1") in site 29.
result = nairobi.aggregate(
[
    {"$match":{"metadata.site":29}},
    {"$group":{"_id":"$metadata.measurement","count":{"$count":{}}}}
]
)
pp.pprint(list(result))

[ {'_id': 'P2', 'count': 32869},
  {'_id': 'temperature', 'count': 33105},
  {'_id': 'humidity', 'count': 33001},
  {'_id': 'P1', 'count': 32877}]


In [20]:
# Use the find method to retrieve the PM 2.5 readings from site 29. Be sure to limit your results to 3 records only. 
# Since we won't need the metadata for our model, use the projection argument to limit the results to the "P2" and "timestamp" 
# keys only.
result = nairobi.find(
    {"metadata.site":29,"metadata.measurement":"P2"}
    ,projection={"P2":1,"timestamp":1,"_id":0}
)
pp.pprint(result.next())

{'P2': 33.36, 'timestamp': datetime.datetime(2018, 11, 1, 13, 20, 9)}


In [21]:
df = pd.DataFrame(result).set_index('timestamp')
df.head()

,P2
timestamp,
2018-11-01 13:25:18,20.41
2018-11-01 13:30:19,11.74
2018-11-01 13:35:21,19.99
2018-11-01 13:40:24,19.03
2018-11-01 13:45:27,17.84
